In [75]:
import numpy as np
import pandas as pd

from configparser import ConfigParser
config = ConfigParser()
# create your own config.ini in root of project folder to store project configurations
config.read('config.ini')


pathfile = config.get('main', 'dirty_csv')       

df = pd.read_csv(pathfile, nrows=10**6)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)

df.head(20)



,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
0,53.0,32.0,-8.0,1642.0,1650,1545,65.0,NaN,0,NaN,4,10,4.0,1549.0,PIT,205.0,0,209,NaN,10,NaN,DCA,NaN,N443US,7.0,14.0,US,NaN,2002
1,164.0,155.0,-11.0,1754.0,1805,1610,175.0,NaN,0,NaN,4,2,0.0,1610.0,MCI,1072.0,0,109,NaN,12,NaN,MCO,NaN,N755,2.0,7.0,WN,NaN,1999
2,60.0,NaN,15.0,2005.0,1950,1850,60.0,NaN,0,NaN,5,10,15.0,1905.0,CLT,227.0,0,1276,NaN,12,NaN,ATL,NaN,NaN,NaN,NaN,DL,NaN,1993
3,51.0,NaN,-5.0,1818.0,1823,1728,55.0,NaN,0,NaN,4,28,-1.0,1727.0,BNA,200.0,0,961,NaN,9,NaN,MEM,NaN,NaN,NaN,NaN,AA,NaN,1989
4,45.0,29.0,2.0,1120.0,1118,1030,48.0,NaN,0,0.0,1,19,5.0,1035.0,CMH,116.0,0,5873,0.0,6,0.0,CVG,0.0,N785CA,3.0,13.0,OH,0.0,2006
5,49.0,37.0,2.0,1137.0,1135,1048,47.0,NaN,0,NaN,4,2,0.0,1048.0,CLT,156.0,0,353,NaN,1,NaN,MYR,NaN,N934VJ,6.0,6.0,US,NaN,1997
6,61.0,40.0,-3.0,1537.0,1540,1440,60.0,NaN,0,NaN,7,20,-4.0,1436.0,LAW,140.0,0,3281,NaN,7,NaN,DFW,NaN,N286AE,7.0,14.0,MQ,NaN,2008
7,150.0,126.0,-19.0,2015.0,2034,1745,169.0,NaN,0,NaN,4,15,0.0,1745.0,ATL,903.0,0,1521,NaN,10,NaN,PVD,NaN,N919DE,14.0,10.0,DL,NaN,1998
8,115.0,103.0,-5.0,735.0,740,640,120.0,NaN,0,NaN,2,16,0.0,640.0,SEA,689.0,0,1678,NaN,6,NaN,SLC,NaN,N346,2.0,10.0,WN,NaN,1998
9,NaN,NaN,NaN,NaN,730,605,85.0,B,1,0.0,7,10,NaN,NaN,SLC,391.0,0,4117,0.0,4,0.0,DEN,0.0,N705EV,0.0,0.0,EV,0.0,2005


# ----------------------cleaning comments----------------------
- Total number of records > 120 million
- remove all records with more than certain number of NaN values?
- At least remove all records with NaNs for assumed FDS except for the ones where NaN has to be transformed to something (will see this for CarrierDelay below)?

- CarrierDelay has 0.0 or NaN for when there is no delay? Turn NaN into 0?
    - Same goes for every other Delay column. Which delay columns do we want to keep?
- CancellationCode has letter for when Cancelled = 1 and NaN for when not cancelled
    - Change NaNs in CancellationCode to unique not-cancelled code?

- Remove records where TailNum = 0? Is normally several numbers and a letter. Or keep for soft functional dependency?
 

In [10]:
df.describe()

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,SecurityDelay,TaxiIn,TaxiOut,WeatherDelay,Year
count,978757.000000,681360.000000,978757.000000,978783.000000,1000000.000000,1000000.000000,999791.000000,1000000.000000,276290.000000,1000000.000000,1000000.000000,981117.000000,981117.000000,998310.000000,1000000.000000,1000000.000000,276290.000000,1000000.000000,276290.000000,276290.000000,696728.000000,696823.000000,276290.000000,1000000.000000
mean,120.218606,102.854060,7.051868,1492.796174,1490.388786,1333.926632,121.124092,0.018890,3.708991,3.941351,15.735314,8.158946,1347.962749,702.446892,0.002353,1364.394253,4.715715,6.556942,4.099834,0.024033,6.476692,15.218328,0.760534,1998.614886
std,68.704049,71.022941,30.809895,498.580434,493.228099,476.174529,68.105616,0.136137,19.515270,1.990578,8.793015,28.673504,476.983626,551.751271,0.048451,1403.895626,20.071980,3.447362,16.662616,0.954795,22.342821,11.073621,8.904026,6.224338
min,-525.000000,-1419.000000,-1410.000000,1.000000,0.000000,0.000000,-30.000000,0.000000,0.000000,1.000000,1.000000,-1410.000000,1.000000,11.000000,0.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,1987.000000
25%,70.000000,54.000000,-7.000000,1115.000000,1115.000000,928.000000,71.000000,0.000000,0.000000,2.000000,8.000000,-2.000000,933.000000,307.000000,0.000000,450.000000,0.000000,4.000000,0.000000,0.000000,4.000000,10.000000,0.000000,1993.000000
50%,102.000000,84.000000,0.000000,1521.000000,1520.000000,1326.000000,102.000000,0.000000,0.000000,4.000000,16.000000,0.000000,1333.000000,545.000000,0.000000,945.000000,0.000000,7.000000,0.000000,0.000000,5.000000,13.000000,0.000000,1999.000000
75%,151.000000,133.000000,11.000000,1916.000000,1912.000000,1727.000000,152.000000,0.000000,0.000000,6.000000,23.000000,6.000000,1737.000000,937.000000,0.000000,1711.000000,0.000000,10.000000,0.000000,0.000000,7.000000,18.000000,0.000000,2004.000000
max,1812.000000,1570.000000,1435.000000,2717.000000,2400.000000,2359.000000,660.000000,1.000000,1126.000000,7.000000,31.000000,1439.000000,2635.000000,4983.000000,1.000000,9619.000000,613.000000,12.000000,875.000000,128.000000,1460.000000,1435.000000,958.000000,2008.000000


# check strong FD (TailNum) -> (UniqueCarrier)

In [76]:
# ---------------- check strong FD (TailNum) -> (UniqueCarrier)---------------
# dependency is not a bi-implication
df_uniq_carr = df[['TailNum','UniqueCarrier']]
df_uniq_carr = df_uniq_carr.sort_values(by=['TailNum'])
df_uniq_carr.head(20)
# describe for for example unique value counts
# df_uniq_carr.describe()

,TailNum,UniqueCarrier
864543,-N037M,MQ
425218,-N037M,MQ
860027,-N037M,MQ
844197,-N047M,MQ
994545,-N047M,MQ
454325,-N047M,MQ
119211,-N047M,MQ
798038,-N047M,MQ
162424,-N047M,MQ
690297,-N107D,DL


In [88]:
# check if dependency also works other way around. Seems not.
# Just pick one uniquecarrier value and filter NaNs on TailNum side
df_uniq_carr_filt = df_uniq_carr[(df_uniq_carr['UniqueCarrier'] == '9E') & (df_uniq_carr['TailNum'].notnull())]
# df_uniq_carr_filt.describe()
df_uniq_carr_filt.tail(50)

,TailNum,UniqueCarrier
2146,91709E,9E
254277,91709E,9E
403016,91709E,9E
910415,91709E,9E
138319,91709E,9E
70986,91709E,9E
42137,91709E,9E
222172,91709E,9E
804150,91709E,9E
160478,91709E,9E


# check strong FD (Origin, Dest) -> (Distance)

In [112]:
df_origin_dest = df[['Origin', 'Dest', 'Distance']]
# df_origin_dest = df_origin_dest.sort_values(by=['Origin', 'Dest'])
df_origin_dest = df_origin_dest.sort_values(by=['Distance','Origin', 'Dest'])
# filter NaN
df_origin_dest[df_origin_dest['Distance'].notnull()].tail(500)
# compare amount of different groups of [origin, dest, distance] (full FD) with groups of [origin, dest] (left side FD)
# should be same number of groups for strong dependency
df_origin_dest.groupby(['Origin', 'Dest', 'Distance']).size().reset_index().rename(columns={0:'count'}).count()

Origin      6775
Dest        6775
Distance    6775
count       6775
dtype: int64

In [113]:
# counts for only left hand side FD
df_origin_dest.groupby(['Origin', 'Dest']).size().reset_index().rename(columns={0:'count'}).count()

Origin    6775
Dest      6775
count     6775
dtype: int64

# check Soft FD (CRSDepTime, DepTime) -> (DepDelay)

In [71]:
df_crsdep_dep = df[['CRSDepTime', 'DepTime', 'DepDelay']]
df_crsdep_dep = df_crsdep_dep.sort_values(by=['CRSDepTime', 'DepTime'])
# df_crsdep_dep.head(500)
df_crsdep_dep.groupby(['CRSDepTime', 'DepTime', 'DepDelay']).size().reset_index().rename(columns={0:'count'}).head(100)
# df_crsdep_dep.count()
# CRSDepTime does not seem to be set on some occations (set to 0). Means that when CRSDepTime is set to 0 and DepTime are equal the DepDelay might differ.
df_crsdep_dep[df_crsdep_dep['CRSDepTime'] == 0].count()

# Check to see if there might be a strong FD when CRSDepTime is actually set
df_crsdep_dep[df_crsdep_dep['CRSDepTime'] != 0].head(50)
# Sometimes DepTime - CRSDepTime != DepDelay even when CRSDepTime is set (example: row 2) -> Soft FD?

,CRSDepTime,DepTime,DepDelay
0,1545,1549.0,4.0
1,1610,1610.0,0.0
2,1850,1905.0,15.0
3,1728,1727.0,-1.0
4,1030,1035.0,5.0
5,1048,1048.0,0.0
6,1440,1436.0,-4.0
7,1745,1745.0,0.0
8,640,640.0,0.0
9,605,NaN,NaN


# check Soft FD (ActualElapsedTime, TaxiIn, TaxiOut) -> (AirTime)

In [102]:
# ---------------- check Soft FD (ActualElapsedTime, TaxiIn, TaxiOut) -> (AirTime)---------------
df_air_time = df[['ActualElapsedTime', 'TaxiIn', 'TaxiOut', 'AirTime']]
df_air_time = df_air_time.sort_values(by=['ActualElapsedTime', 'TaxiIn', 'TaxiOut'])
df_air_time.head(50)

# Take a look at specific rows that have equal left hand side but AirTime is sometimes different.
df.iloc[[214815, 328863, 689521, 867367]]

# yet another few examples with same left hand side but sometimes different AirTime. Seems to be connection with CRSElapsedTime?
df.iloc[[119130, 615929, 673863]]


,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
119130,16.0,9.0,7.0,1056.0,1049,1029,20.0,NaN,0,NaN,1,27,11.0,1040.0,PSG,31.0,0,65,NaN,5,NaN,WRG,NaN,N747AS,3.0,4.0,AS,NaN,2002
615929,16.0,12.0,-7.0,1248.0,1255,1232,23.0,NaN,0,0.0,5,14,0.0,1229.0,VIS,36.0,0,6055,0.0,5,0.0,FAT,0.0,N292UX,3.0,4.0,OO,0.0,2004
673863,16.0,9.0,-2.0,1358.0,1400,1340,20.0,NaN,0,NaN,5,15,2.0,1342.0,ITH,32.0,0,1175,NaN,11,NaN,ELM,NaN,N886US,3.0,4.0,US,NaN,1996


# check Soft FD (AirTime, TaxiIn, TaxiOut) -> (ActualElapsedTime)

In [134]:
df_air_time_delay = df[['AirTime', 'TaxiIn', 'TaxiOut', 'ActualElapsedTime']]
df_air_time_delay = df_air_time_delay.sort_values(by=['AirTime', 'TaxiIn', 'TaxiOut'])
df_air_time_delay[df_air_time_delay['AirTime'].notnull() & df_air_time_delay['TaxiIn'].notnull() & df_air_time_delay['TaxiOut'].notnull()].head(500)





,AirTime,TaxiIn,TaxiOut,ActualElapsedTime
139136,-1419.0,1441.0,20.0,42.0
633330,-1417.0,1443.0,14.0,40.0
708954,-1414.0,1444.0,7.0,37.0
773258,-1413.0,1442.0,13.0,42.0
140584,-1413.0,1444.0,17.0,48.0
464072,-1410.0,1444.0,15.0,49.0
211788,-1410.0,1444.0,16.0,50.0
323928,-1410.0,1445.0,20.0,55.0
951785,-1409.0,1442.0,8.0,41.0
922405,-1409.0,1445.0,8.0,44.0


In [133]:
df_air_time_delay.groupby(['AirTime', 'TaxiIn', 'TaxiOut']).count().reset_index().rename(columns={0:'count'})


,AirTime,TaxiIn,TaxiOut,ActualElapsedTime
0,-1419.0,1441.0,20.0,1
1,-1417.0,1443.0,14.0,1
2,-1414.0,1444.0,7.0,1
3,-1413.0,1442.0,13.0,1
4,-1413.0,1444.0,17.0,1
...,...,...,...,...
130236,630.0,8.0,14.0,1
130237,637.0,6.0,26.0,1
130238,655.0,4.0,19.0,1
130239,1562.0,4.0,66.0,1


In [132]:
df_air_time_delay.groupby(['AirTime', 'TaxiIn', 'TaxiOut', 'ActualElapsedTime']).count().reset_index().rename(columns={0:'count'})
# only +- 6.000 different groups between the two so soft dependency?

,AirTime,TaxiIn,TaxiOut,ActualElapsedTime
0,-1419.0,1441.0,20.0,42.0
1,-1417.0,1443.0,14.0,40.0
2,-1414.0,1444.0,7.0,37.0
3,-1413.0,1442.0,13.0,42.0
4,-1413.0,1444.0,17.0,48.0
...,...,...,...,...
136064,630.0,8.0,14.0,652.0
136065,637.0,6.0,26.0,669.0
136066,655.0,4.0,19.0,678.0
136067,1562.0,4.0,66.0,1812.0


In [135]:
# Take a look at specific rows that have equal left hand side but AirTime is sometimes different.
df.iloc[[494368, 508073, 645262, 968918]]

,ActualElapsedTime,AirTime,ArrDelay,ArrTime,CRSArrTime,CRSDepTime,CRSElapsedTime,CancellationCode,Cancelled,CarrierDelay,DayOfWeek,DayofMonth,DepDelay,DepTime,Dest,Distance,Diverted,FlightNum,LateAircraftDelay,Month,NASDelay,Origin,SecurityDelay,TailNum,TaxiIn,TaxiOut,UniqueCarrier,WeatherDelay,Year
494368,17.0,9.0,64.0,1837.0,1733,1714,19.0,NaN,0,NaN,3,17,66.0,1820.0,WRG,31.0,0,64,NaN,6,NaN,PSG,NaN,N744AS,3.0,5.0,AS,NaN,1998
508073,24.0,9.0,0.0,2143.0,2200,2136,24.0,NaN,0,NaN,5,17,0.0,2126.0,SGU,50.0,0,3622,NaN,1,NaN,CDC,NaN,N308SW,3.0,5.0,OO,NaN,2003
645262,17.0,9.0,6.0,1301.0,1255,1232,23.0,NaN,0,0.0,5,9,12.0,1244.0,VIS,36.0,0,6055,0.0,1,0.0,FAT,0.0,N292UX,3.0,5.0,OO,0.0,2004
968918,17.0,9.0,88.0,1200.0,1032,1010,22.0,NaN,0,0.0,6,9,93.0,1143.0,PSG,31.0,0,65,88.0,7,0.0,WRG,0.0,N783AS,3.0,5.0,AS,0.0,2005
